<a href="https://colab.research.google.com/github/thislis/XAI_Sentiment_Analysis/blob/main/R%26E_main_anger_lstm2028.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

anger 약 20.28% 정확도 0.8403

코드 참조
1. https://keras.io/api/layers/recurrent_layers/bidirectional/


In [ ]:
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#텍스트 전처리
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
from collections import Counter
from konlpy.tag import Mecab
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

#모델링
import re
from tensorflow.keras.layers import Embedding, Dense, LSTM, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [4]:
train1 = pd.read_csv('/content/drive/Shareddrives/R&E/dataset/train_anger.csv', names=['감정', '문장'], skiprows=1)
train2 = pd.read_csv('/content/drive/Shareddrives/R&E/dataset/train_anxiety.csv', names=['감정', '문장'], skiprows=1)
train3 = pd.read_csv('/content/drive/Shareddrives/R&E/dataset/train_embarrassment.csv', names=['감정', '문장'], skiprows=1)
train4 = pd.read_csv('/content/drive/Shareddrives/R&E/dataset/train_happy.csv', names=['감정', '문장'], skiprows=1)
train5 = pd.read_csv('/content/drive/Shareddrives/R&E/dataset/train_happy.csv', names=['감정', '문장'], skiprows=1)
train6 = pd.read_csv('/content/drive/Shareddrives/R&E/dataset/train_sadness.csv', names=['감정', '문장'], skiprows=1)
train_anger_data = pd.concat([train1, train2, train3, train4, train5, train6], ignore_index=True)

train_anger_data.drop_duplicates(subset=['문장'], inplace=True) #중복 제거
def t_2_01(text):
  if text == "분노":
    return 1
  else:
    return 0
train_anger_data['감정'] = train_anger_data['감정'].apply(t_2_01)

print('전체 문장 개수 :',len(train_anger_data))
print(len(train_anger_data.loc[train_anger_data['감정'] == 1]))

전체 문장 개수 : 182369
36995


In [5]:
test_anger_data = pd.read_csv('/content/drive/Shareddrives/R&E/dataset/test_dataset.csv', names=['감정', '문장'], skiprows=1)
test_anger_data.drop_duplicates(subset=['문장'], inplace=True) #중복 제거
def t_2_01(text):
  if text == "분노":
    return 1
  else:
    return 0
test_anger_data['감정'] = test_anger_data['감정'].apply(t_2_01)
#print(test_anger_data)

In [6]:
train_anger_data['문장'] = train_anger_data['문장'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") #정규 표현식 수행
train_anger_data['문장'].replace('', np.nan, inplace=True) #공백은 Null 값으로 변경
#print(train_anger_data.isnull().sum())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [7]:
test_anger_data['문장'] = test_anger_data['문장'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
test_anger_data['문장'].replace('', np.nan, inplace=True)
test_anger_data = test_anger_data.dropna(how='any')
#print(len(test_anger_data))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [8]:
anger_data = pd.concat([train_anger_data, test_anger_data], ignore_index=True)
train_anger_data, test_anger_data = train_test_split(anger_data, test_size = 0.25, random_state = 42)
print('훈련용 :', len(train_anger_data))
print('테스트용 :', len(test_anger_data))

훈련용 : 158030
테스트용 : 52677


In [9]:
stopwords = ['도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게', '만', '되', '음', '면']

In [10]:
mecab = Mecab() 

train_anger_data['tokenized'] = train_anger_data['문장'].apply(mecab.morphs)
train_anger_data['tokenized'] = train_anger_data['tokenized'].apply(lambda x: [item for item in x if item not in stopwords])
test_anger_data['tokenized'] = test_anger_data['문장'].apply(mecab.morphs)
test_anger_data['tokenized'] = test_anger_data['tokenized'].apply(lambda x: [item for item in x if item not in stopwords])

In [11]:
positive_words = np.hstack(train_anger_data[train_anger_data.감정 == 1]['tokenized'].values)
positive_word_count = Counter(positive_words)
#print(positive_word_count.most_common(20))

In [12]:
X_train = train_anger_data['tokenized'].values
y_train = train_anger_data['감정'].values
X_test= test_anger_data['tokenized'].values
y_test = test_anger_data['감정'].values

In [13]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [14]:
threshold = 2
total_cnt = len(tokenizer.word_index) #단어 수
rare_cnt = 0 #등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 #훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 #등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

#(단어, 빈도수)쌍(pair)을 key와 value로 받음.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

vocab_size = total_cnt - rare_cnt + 2

In [15]:
tokenizer = Tokenizer(vocab_size, oov_token = 'OOV') 
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [16]:
#def below_threshold_len(max_len, nested_list):
#  count = 0
#  for sentence in nested_list:
#    if(len(sentence) <= max_len):
#        count = count + 1
#  print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (count / len(nested_list))*100))
#below_threshold_len(max_len, X_train)

max_len = 35

In [17]:
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

In [18]:
embedding_dim = 100
hidden_units = 128

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(Bidirectional(LSTM(hidden_units))) # Bidirectional LSTM
model.add(Dense(1, activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=128, validation_split=0.2)

Epoch 1/15
983/988 [============================>.] - ETA: 0s - loss: 0.4166 - acc: 0.8311
Epoch 1: val_acc improved from -inf to 0.83671, saving model to best_model.h5
988/988 [==============================] - 19s 11ms/step - loss: 0.4164 - acc: 0.8312 - val_loss: 0.4013 - val_acc: 0.8367
Epoch 2/15
988/988 [==============================] - ETA: 0s - loss: 0.3862 - acc: 0.8419
Epoch 2: val_acc improved from 0.83671 to 0.83927, saving model to best_model.h5
988/988 [==============================] - 9s 10ms/step - loss: 0.3862 - acc: 0.8419 - val_loss: 0.3937 - val_acc: 0.8393
Epoch 3/15
987/988 [============================>.] - ETA: 0s - loss: 0.3762 - acc: 0.8464
Epoch 3: val_acc improved from 0.83927 to 0.84212, saving model to best_model.h5
988/988 [==============================] - 10s 10ms/step - loss: 0.3762 - acc: 0.8464 - val_loss: 0.3898 - val_acc: 0.8421
Epoch 4/15
984/988 [============================>.] - ETA: 0s - loss: 0.3691 - acc: 0.8507
Epoch 4: val_acc did not imp

In [19]:
loaded_model = load_model('best_model.h5')
print("테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

1647/1647 [==============================] - 6s 3ms/step - loss: 0.3918 - acc: 0.8403
테스트 정확도: 0.8403
